# **S02 - RISPOSTA IN FREQUENZA E FFT**

L'obiettivo di quest'esperimento è lo studio della risposta in frequenza di un circuito lineare passivo che agisce da filtro passa banda.

# Setting dell'esperimento

Le componenti scelte per realizzare il circuito sono: $R_1 = 100k\Omega$ (tolleranza 1%) , $R_2 = 1k\Omega$ (tolleranza 1%), $C_1 = 100 nF$ , $C_2 = 47 nF$. Si sono collegati in serie un passa-alto realizzato con $R_1$ e $C_1$ e un passa-basso realizzato con $R_2$ e $C_2$; il canale 1 misura il potenziale a capo di $C_1$, mentre il canale 2 misura il potenziale a capo di $C_2$.

Come spiegato in seguito, serve avere delle misure delle resistenze eseguite con multimetro. Queste sono:


$R_1 = 100.47 \pm 0.06 \, k\Omega$

$R_2 = 0.9920 \pm 0.0006 \, k\Omega$


# Cenni teorici

Ricordiamo di seguito le nozioni fondamentali.

La funzione di trasferimento di un passa banda costituito da passa alto più passa basso a cascata è:

$ H(\omega) = \frac{j\omega\tau_{1}}{(1 + j\omega\tau_{1})(1 + j\omega \tau_{2} )+j\omega R_1 C_2}$

Le frequenze angolari di cornering $\omega_a = 1/\tau_a$ e $\omega_b = 1/\tau_b$ si ricavano trovando le radici del polinomio di secondo grado al denominatore: in corrispondenza di queste il quadagno è pari a $-3 dB$.


# Simulazione numerica della funzione di guadagno e Grafico di Bode

A questo punto sono state realizzate due simulazioni numeriche della funzione guadagno del filtro, una tramite MATLAB ($\verb|bode_plot.m|$) e l'altra tramite Tina ($\verb|bode_tina.TSC|$ e $\verb|data_tina.txt|$), con lo scopo di confrontare i risultati ottenuti.

Di seguito il plot di confronto ($\verb|Simulazioni_CRRC.pdf|$):

<img src="Simulazioni_CRRC.png" Width="600" />

Notiamo che i risultati sono sostanzialmente identici, come ci aspettavamo.



# **Osservazioni sullo script dac+adc**

In questa parte si è utilizzato lo script $\texttt{dac+adc.ipynb}$ con il quale viene mandato un segnale attraverso il circuito e si osservano le registrazioni dei due canali al variare di parametri quali frequenza, sampling, ampiezza, forma d'onda e così via. 

## Forma d'onda sinusoidale
Ampiezza del segnale: 2 V
* a $\omega$ = 10 rad/s il segnale nel canale 2 è fortemente attenuato e vediamo uno sfasamento ad occhio approssimabile a $\pi/2$
* a $\omega$ = 100 rad/s notiamo un guadagno ad occhio poco maggiore di $1/2$ e che i due segnali sono sfasati
* a $\omega$ = 1000 rad/s, in quella che si pensa essere la banda di passaggio del filtro, il guadagno non è unitario ma pari ad occhio un fattore $0.7$
* a $\omega$ = 10^5 rad/s il segnale nel canale torna ad essere fortemente attenuato e sfasato di circa $\pi/2$

N.B.: non si riescono a trovare frequenze per le quali il guadagno è circa unitario

## Forma d'onda tringolare e quadrata
Facendo variare $\omega$ su diverse decadi notiamo:
* A frequenze "basse" i segnali vengono "derivati" (siamo nella zona con slope positiva, dove agisce il passa-alto): in uscita le onde quadre diventano una successione di picchi e le onde triangolari diventano a mano a mano onde quadre. 
* Per frequenze "alte" i segnali vengono "integrati" (zona con slope negativa, agisce il passa basso): in uscita le onde quadre diventano onde triangolari, mentre le onde triangolari diventano delle successioni di archi di parabola

Le osservazioni sullo sfasamento e sul guadagno sono analoghe al caso sinusoidale: allontanandosi della banda di passaggio abbiamo guadagni sempre minori e sfasamenti sempre più tendenti a $\pi/2$.

## Impulsi gaussiani
Se mandiamo nel circuito degli impulsi gaussiani notiamo che in uscita troviamo sempre un impulso della stessa forma: il guadagno non sembra cambiare al variare della larghezza della campana.



# **Osservazioni su script sweepfreq**

* La spaziatura logaritmica tra $f_0$ a $f_1$ è stata creata tramite un logspace che va da $\log{f_0}$ a $\log{f_1}$.
* La frequenza di sampling viene scelta in modo da campionare nel numero di periodi richiesto il maggior numero di punti entro il limite imposto, quindi secondo la relazione:

$f_s = \frac{ff\times npt}{nper} = \frac{npt}{T\times nper}$

con T il periodo del segnale.

N.B.: $f_s$ può essere solo un sottomultiplo di 100 MHz, per cui il risultato è arrotondato per eccesso.

* A ogni acquisizione (quindi per ogni frequenza nel nostro intervallo di spazzata) ampiezza e fase dei due segnali viene calcolata tramite fit: queste però devono essere rispettivamente positiva e contenuta nell'intervallo $[-\pi;\pi]$. Il problema è risolto da un'istruzione $\texttt{if-else}$ che itera il fit fino a quando i parametri di best fit non soddisfano la nostra richiesta.

# **Fit della funzione di guadagno e dello sfasamento**

## Acquisizione dei dati e Calcolo delle incertezze

Il primo passo è stata l'acquisizione dei dati di guadagno e sfasamento tramite lo script $\texttt{sweepfreq.ipynb}$ che ci ha permesso di analizzare la risposta del circuito a diverse frequenze.

Per poter stimare gli errori si è deciso di eseguire il campionamento delle misure per cinque volte così a effettuare, punto per punto, media e deviazione standard. Quest'ultimi sono i punti e le incertezze utilizzati per il fit ($\verb|data_CRRC_final.txt|$).


## Esecuzione del fit

Il fit è stato eseguito tramite il file di MATLAB $\texttt{fit CRRC.m}$ con due diverse funzioni modello.

Nel primo caso è stata utilizzata la funzione:

$ |H(\omega)| = \left|\frac{j\omega X}{(1 + j\omega X)(1 + j\omega Y )+j\omega Z}\right|$

con $X = \tau_1$ , $Y = \tau_2$ e $Z = R_1 C_2$ parametri di fit, con lo scopo di ricavare da tali parametri i valori delle capacità usate nel circuito.

Il secondo fit è stato eseguito con la funzione:

$ |H(\omega)| = \left|\frac{j\omega R}{(1 + j\omega \frac{1}{S})(1 + j\omega \frac{1}{T})}\right|$

on $R = \tau_1$ , $S = \omega_a$ e $T = \omega_b$ parametri di fit. Questo seconda funzione di fit è stata scelta con lo scopo di ricavare direttamente le frequenze angolari di cornering $\omega_a$ e $\omega_b$ da confrontare con i valori calcolati.

## Risultati ottenuti

Dal primo fit sono stati ricavati i seguenti valori che confrontiamo con quelli attesi.

| |Valore ricavato dal fit|Valore atteso/calcolato| 
|:---:|:---:|:---:| 
|$\tau_1 = R_1 C_1$|$(9.206 \pm 0.001)\times 10^{-3}$ s|$10^{-2}$ s|
|$\tau_2 = R_2 C_2$|$(4.4947 \pm 0.0001)\times 10^{-5}$ s|$4.7 \times 10^{-5}$ s|
|$R_1 C_2$|$(4.2391 \pm 0.0006)\times 10^{-3}$ s|$4.7\times 10^{-3}$ s|
|$\chi^2_{rid}$|$1945$|$1$|

Ricaviamo ora i valori delle capacità utilizzate (dove per $R_1$ e $R_2$ si utilizzano le misure con il multimetro e per $\tau_1$ e $\tau_2$ i valori el fit).

| |Valore ricavato|Valore nominale| 
|:---:|:---:|:---:| 
|$C_1 = \frac{\tau_1}{R_1}$|$91.60 \pm 0.09$ nF|$100$ nF|
|$C_2 = \frac{\tau_2}{R_2}$|$45.31 \pm 0.05$ nF|$47$ nF|

Dal secondo fit invece ricaviamo le frequenze angolari di corner $\omega_a$ e $\omega_b$.

| |Valore ricavato dal fit|Valore atteso/calcolato| 
|:---:|:---:|:---:| 
|$\omega_a$|$74.30 \pm 0.01$ rad/s|$67.96$ rad/s|
|$\omega_b$|$32527.5 \pm 0.5$ rad/s|$31309$ rad/s|

Per tutte le misure si può osservare che i valori non sono compatibili entro le incertezze di misura, le quali sono molto piccole. Questa non compatibilità si conferma nel valore del $\chi^2$ ridotto, il quale è vertiginosamente alto.

Il guadagno a centro banda è stato stimato attraverso il massimo dell'array del guadagno ed ha il valore di $G_{max} \simeq 0.68$.

Di seguito il grafico del fit e dei residui. Dai residui è evidente una forte correlazione tra le incertezze e la probabile presenza di un errore sistematico.

<img src="fit_CRRC.png" Width="500" />
<img src="residui_CRRC.png" Width="500" />

# **Trasformata di Fourier**

## Acquisizione di un segnale a dente di sega

Come prima cosa, utilizzando lo script $\texttt{dac+adc.ipynb}$, è stata generata e acquisita un'onda a dente di sega, ottenuta da una forma d'onda triangolare con duty cycle 100%. Questa operazione è stata iterata per 4 volte facendo variare la frequenza di sampling $f_s$ per diversi valori (scelti con criterio e sotto specificati) e mantenendo invece fissa la frequenza del segnale a $10 \, kHz$. Ognuna di queste acquisizioni è stata salvata in un diverso file di testo $\verb|data_rampup_[i].txt|$.

Dopodichè è stato scritto un codice Matlab ($\verb|FFT_rampup.m|$) che calcola e plotta insieme il segnale in funzione del tempo e la sua trasformata di Fourier discreta, in modo da osservare come cambia il risultato al variare di $f_s$

Di seguito alcune osservazioni sui grafici ottenuti:

* **$f_s = 1$ MSa/s** : data l'elevata $f_s$ osserviamo un segnale ben ricostruito e con una trasformata ragionevole, i cui picchi decrescono in funzione della frequenza (come 1/k secondo lo sviluppo in coefficienti di Fourier).
* **$f_s = 38$ kSa/s** : osserviamo che l'onda non è acquisita al meglio in quanto la frequenza di sampling non è un multiplo della frequenza del segnale, e ciò comporta che l'acquisizione di ogni periodo avviene con un ritardo diverso ed il segnale appare "storto". Per quanto riguarda la trasformata vediamo un picco a $10^4$ Hz, ovvero quello del segnale e dei picchi più piccoli a destra e sinistra che sono manifestazione dell'aliasing.
* **$f_s = 10$ kSa/s** : qua la frequenza di campionamento e quella del segnale sono uguali, per cui il segnale appare praticamente piatto (per la precisione oscilla "poco", nell'ordine del millesimo di volt). Nella trasformata vediamo un "picco" a frequenza 0 (che in realtà sarebbe l'offset del segnale) e nessun picco a seguire, come ci si aspetterebbe.
* **$f_s = 10.5$ kSa/s**: qua osserviamo non solo che il segnale è "sballato" ma anche che la trasformata dei picchi che decrescono similmente a come avviene nel primo caso. Purtroppo è uno spettro "fittizio": la "poca" differenza tra le due frequenze (del segnale e di sampling) fa si che ci sia un fenomeno di battimenti, come testimonia il fatto che il picco più alto si trova a $500 \, Hz$ (infatti $10500-10000 = 500$)

Nel caso della frequenza di sampling $f_s = 38$ kSa/s è evidente il **fenomeno dell'Aliasing**, il quale prevede il ripetersi dei picchi della trasformata in corrispondenza di un multiplo intero della frequenza di sampling. Questo avviene in quanto non è rispettato il Criterio di Nyquist ($f_s > 2.5 f_{max}$) dalla seconda armonica in poi. Di seguito il grafico che evidenzia il fenomeno:

<img src="Aliasing.png" Width="500" />

## Leakage spettrale

Il leakage spettrale occorre quando la frequenza del segnale $f_0$ non ricade in nessuna delle $f_m = \frac{m\times f_s}{N}$, con N numero di punti; in altre parole, affinchè i picchi della trasformata non si "sparpaglino" il segnale deve essere acquisito per un numero intero di periodi. 

Sempre con un'onda a dente di sega, sono state fatte due acquisizioni a N fissato ($N=5000$) ma con due frequenze di sampling diverse, rispettivamente $3.75$ e $5 \, MSa/s$; nel primo caso si hanno un numero semi-intero di periodi, nel secondo un numero intero. Il risultato è che nel primo segnale si verifica leakage, ma solo per i multipli dispari dell'armonica fondamentale (per i multipli pari è facile convincersi che la condizione detta all'inizio è soddisfatta). Nel secondo segnale invece non si ha leakage.

Di seguito si trova un grafico che confronta le due trasformate:

<img src="Leakage.png" Width="500" />

## Funzione di risposta: studio impulsato

In quest'ultima parte dell'esperimento, utilizzando lo script $\texttt{dac+adc.ipynb}$, si è acquisito sia $V_{in}(t)$ che $V_{out}(t)$ di un'impulso, generato tramite una forma d'onda quadra con duty cycle molto piccolo. Successivamente, nel file $\verb|MisuraImpulsata_CRRC.m|$, si è plottata la funzione di risposta del circuito, calcolando il modulo del rapporto tra le trasformate di $V_{out}(t)$ e $V_{in}(t)$. 

Inoltre sono state aggiunte delle righe di codice che permettono di eliminare una porzione di rumore dal segnale $\verb|Hv(abs(fft(tbl.Vin))<50)=NaN;|$.

Per ottimizzare al meglio la misura si sono scelti i seguenti parametri:

* Frequenza del segnale: 1 Hz (così da prendere almeno un tratto della slope positiva)
* Duty cycle onda quadra: 0.05
* Ampiezza segnale = 4.8 V
* Frequenza di sampling: 0.5 Msa/s (scelta così lenta per permettere il trasferimento dei dati)
* Numero punti = 10000

Il grafico finale prodotto ($\verb|MisuraImpulsata_CRRC.png|$):

<img src="MisuraImpulsata_CRRC.png" Width="500" />

Dal grafico è possibile notare che il plot risultante ha una forma simile alla funzione di guadagno misurata in precedenza, ma non si è riusciti a ricostruire la zona a sinistra del primo corner, quella con slope positiva, della quale è possibile osservare solo un breve tratto.

Infine la zona con slope negativa rimane ancora molto rumorosa, nonostante l'ottimizzazione dei parametri.